In [13]:
import os
import yaml
import nbimporter
from datetime import datetime, date
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType,  DateType, TimestampType
from pyspark.sql.functions import to_date, to_timestamp
from lab_table_manager import TableManager
import yfinance as yf
from lab_pg_database_manager import PGDatabaseManager

In [14]:
with open("cfg_connections.yaml","r") as file:
    config=yaml.safe_load(file)
    catalog_uri = config['docker_env']['catalog_uri'] 
    warehouse = config['docker_env']['warehouse']     # Minio Address to Write to
    storage_uri = config['docker_env']['storage_uri'] # Minio IP address from docker inspec

# Configure Spark with necessary packages and Iceberg/Nessie settings
conf = (
    pyspark.SparkConf()
        .setAppName('finalytics_app')
        # Include necessary packages
        .set('spark.jars.packages',
             'org.postgresql:postgresql:42.7.3,'
             'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,'
             'org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.77.1,'             
             # awssdk 2.29.42 compatible with spark 3.5.4
             'software.amazon.awssdk:bundle:2.29.42,'
             'software.amazon.awssdk:url-connection-client:2.29.42')
        # Enable Iceberg and Nessie extensions
        .set('spark.sql.extensions', 
             'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,'
             'org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        # Configure Nessie catalog
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', catalog_uri)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        # Set Minio as the S3 endpoint for Iceberg storage
        .set('spark.sql.catalog.nessie.s3.endpoint', storage_uri)
        .set('spark.sql.catalog.nessie.warehouse', warehouse)
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
        # Set master location, the job will be sent to the cluster
        .set('spark.master', 'spark://172.33.0.30:7077')
)   

# Start Spark session
spark = SparkSession.builder.config(conf=conf).getOrCreate()  
# Create the "raw" namespace
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.raw;")



DataFrame[]

In [15]:
def fetch_yfinance_record(symbol_date_pairs):
    try:
        symbol, start_date = symbol_date_pairs
        # Fetch stock data using yfinance
        quote = yf.Ticker(symbol)
        current_date = date.today()
        hist = quote.history(start=start_date, end=current_date)

        # Reset index to include Date as a column and format it
        hist.reset_index(inplace=True)
        hist["Date"] = hist["Date"].dt.date
        
        # limit and stablize the fields of hist
        hist = hist[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']]
        
        # Add symbol and import_time to each row
        record_list = [
            tuple(row) + (symbol, import_time) for row in hist.itertuples(index=False)
        ]

        # print(record_list)
        return record_list

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return []  # Return an empty list on error

In [16]:
def parallel_fetch_yfinance_record(symbol_date_pairs, record_schema):
    try:
        # Distribute (symbol, start_date) pairs across Spark workers
        record_rdd = spark.sparkContext.parallelize(symbol_date_pairs)

        
        # Fetch data in parallel
        mapped_record_rdd = record_rdd.flatMap(fetch_yfinance_record)

        # Convert RDD to DataFrame
        result_df = spark.createDataFrame(mapped_record_rdd, schema=record_schema)
        # result_df = spark.createDataFrame(mapped_record_rdd)

        # Show or save the results
        # result_df.show()
        return result_df
    except Exception as e:
        print(f"Error paralleling fetch: {e}")
        return spark.createDataFrame([])

In [17]:
def load_iceberg_raw_eod_yfinance(symbol_date_pairs, iceberg_sink_table, schema_config_file):
    try: 
        table_manager=TableManager(schema_config_file)
        schema_struct_type=table_manager.get_struct_type(iceberg_sink_table)   

        # print(schema_struct_type)
        create_table_script = table_manager.get_create_table_query(iceberg_sink_table)
        spark.sql(create_table_script)

        df_source=parallel_fetch_yfinance_record(symbol_date_pairs, schema_struct_type)
        
        # df_source.writeTo(iceberg_sink_table).append()
        df_source.write.mode("overwrite").saveAsTable(iceberg_sink_table)    
        # df_source.writeTo(iceberg_sink_table).overwritePartitions() 

        print(f"{iceberg_sink_table} has been loaded")
        
    except Exception as e:
        print(f"Error loading lceberg raw table: {e}")
        

In [18]:
def load_pg_finalytics(iceberg_source_table, pg_url, pg_driver, pg_sink_table, pg_truncate_script, pg_merge_script):   
    try:     
        df_source=spark.read.table(iceberg_source_table)          
        finalytics.execute_sql_script(pg_truncate_script)
        
        # Write DataFrame to PostgreSQL
        df_source.write.jdbc(url=pg_url, table=pg_sink_table, mode="append", properties={"driver": pg_driver})        
        finalytics.execute_sql_script(pg_merge_script)
    except Exception as e:
        print(f"Error loading pg finalytics: {e}")        


## Load nessie.raw.stock_eod_yfinance

In [19]:
# symbol_start_date_pairs = [
#     ('TSLA', '2024-11-20'),
#     ('AAPL', '2024-11-20'),
#     ('C', '2024-11-20'),
# ]

# Get finalytics connetion info
conn_config_file='cfg_connections.yaml'
finalytics=PGDatabaseManager(conn_config_file, 'finalytics')
pg_url=finalytics.jdbc_url
pg_driver=finalytics.driver

# Get symbol_start_date_pairs from finalytics
query="select symbol, start_date from fin.vw_etl_stock_eod_start_date limit 20"
query="select symbol, start_date from fin.vw_etl_stock_eod_start_date"
symbol_start_date_pairs=finalytics.get_sql_script_result_list(query)
print(symbol_start_date_pairs)


# Get iceberg table config info
table_schema_config_file='cfg_table_schemas.yaml'
iceberg_table='nessie.raw.stock_eod_yfinance'

# Set global import_time
import_time = datetime.now().isoformat()

# Load nessie.raw.stock_eod_yfinance 
load_iceberg_raw_eod_yfinance(symbol_start_date_pairs, iceberg_table, table_schema_config_file)


nessie.raw.stock_eod_yfinance has been loaded


## Load finalytics.stage.stock_eod_quote_yahoo and merge into fin.stock_eod_quote

In [20]:
pg_table = "stage.stock_eod_quote_yahoo"  # Replace with the PostgreSQL table name

pg_truncate_script = f"truncate table {pg_table};"

pg_merge_script = "call fin.usp_load_stock_eod();"

load_pg_finalytics(iceberg_table, pg_url, pg_driver, pg_table,  pg_truncate_script, pg_merge_script)

In [21]:
spark.sql(f'select count(*) from {iceberg_table}').show()

+--------+
|count(1)|
+--------+
|      78|
+--------+



In [22]:
spark.sql('select * from nessie.raw.stock_eod_yfinance').show(truncate=False)

+----------+------+------+------+------+---------+---------+------------+------+--------------------------+
|date      |open  |high  |low   |close |volume   |dividends|stock_splits|symbol|import_time               |
+----------+------+------+------+------+---------+---------+------------+------+--------------------------+
|2024-11-20|345.0 |346.6 |334.3 |342.03|66340700 |0.0      |0.0         |TSLA  |2024-12-30T18:20:14.366923|
|2024-11-21|343.81|347.99|335.28|339.64|58011700 |0.0      |0.0         |TSLA  |2024-12-30T18:20:14.366923|
|2024-11-22|341.09|361.53|337.7 |352.56|89140700 |0.0      |0.0         |TSLA  |2024-12-30T18:20:14.366923|
|2024-11-25|360.14|361.93|338.2 |338.59|95890900 |0.0      |0.0         |TSLA  |2024-12-30T18:20:14.366923|
|2024-11-26|341.0 |346.96|335.66|338.23|62295900 |0.0      |0.0         |TSLA  |2024-12-30T18:20:14.366923|
|2024-11-27|341.8 |342.55|326.59|332.89|57896400 |0.0      |0.0         |TSLA  |2024-12-30T18:20:14.366923|
|2024-11-29|336.08|345.45|33

In [23]:
# print(symbol_start_date_pairs)